In [1]:
pip install selenium

In [2]:
pip install webdriver_manager

In [3]:
pip install urllib3

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install keyboard

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options
import keyboard
from keyboard import press
import json
import urllib.parse
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
class Reports:
    def __init__(self):
        self.filtered_result=None
        self.output=None
        self.raw_data=input('Please input the working directory of your raw data:')
        self.minimum_avgR3M=float(input('Please input your desired avg_R3M:'))
        self.minimum_no_of_users=int(input('Please input your desired minimun users of each country:'))
        self.chrome_options = Options()
        self.chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
        self.chrome_driver = ChromeDriverManager().install()  # install directly
        self.driver = webdriver.Chrome(self.chrome_driver, chrome_options=self.chrome_options)
    def filter_users(self):

        df = pd.read_csv(self.raw_data)
        df['AvgMonReport_R3M'] = df['AvgMonReport_R3M'].fillna(0)
        df.loc[(df['DistributorID'] == 18518), 'filter'] = 'AW direct customer'
        df.loc[(df['DistributorID'] == 10425) , 'filter'] = 'TW'
        df.loc[(df['Country'] == 'KR'), 'filter'] = 'KR'
        df.loc[df['Country'] == 'CN', 'filter'] = 'CN'
        df = df.set_index('filter')

        direct = df[(df['DistributorID'] == 18518)]
        TW = df[(df['DistributorID'] == 10425) & (df['AvgMonReport_R3M'] >= self.minimum_avgR3M)]
        KR = df[(df['Country'] == 'KR') & (df['AvgMonReport_R3M'] >= self.minimum_avgR3M)]
        CN = df[(df['Country'] == 'CN') & (df['AvgMonReport_R3M'] >= self.minimum_avgR3M)]

        groupdict={
            'TW':TW,
            'CN':CN,
            'direct':direct,
            'KR':KR
        }

        for group in groupdict:
            if len(groupdict[group])<self.minimum_no_of_users:
                if group=='KR' or group=='CN':
                    groupdict[group]=df[df['Country']==group].head(self.minimum_no_of_users)
                  
                elif group=='TW':
                    groupdict[group]=df[(df['DistributorID'] == 10425)].head(self.minimum_no_of_users)
                elif group=='direct':
                    groupdict[group]=df[(df['DistributorID'] == 18518)].head(self.minimum_no_of_users)
        self.filtered_result=pd.concat(groupdict.values())
        return self.filtered_result

    def get_repert_link(self):
        df = self.filtered_result
        style_for_CN=input('suffix for CN reports:')
        style_for_KR=input('suffix for KR reports:')
        style_for_TW=input('suffix for TW reports')

        KR = df[df['Country'] == 'KR']
        TW = df[df['Country'] == 'TW']
        CN = df[df['Country'] == 'CN']

        list=['KR','TW','CN']
        country_web = {
            'CN': 'https://datastudio.google.com/u/0/reporting/b9de3989-758e-4c24-920d-c527db9d485c/page/mlqLC',
            'KR': 'https://datastudio.google.com/u/0/reporting/b9de3989-758e-4c24-920d-c527db9d485c/page/lSTHC',
            'TW': 'https://datastudio.google.com/u/0/reporting/b9de3989-758e-4c24-920d-c527db9d485c/page/f2pLC'
        }

        KR_priceplan = {
            'KR_D_Credit 120': 'Credit 120',
            'KR_D_Credit 60': 'Credit 60',
            'KR_D_Credit 300': 'Credit 300',
            'KR_D_Credit 30': 'Credit 30',
            'KR_D_Credit 10': 'Credit 10',
        }
        TW_priceplan = {
            'TW_Credit 600': '600',
            'TW_Credit 300': '300',
            'TW_Credit 240': '240',
            'TW_Credit 120': '120',
            'TW_Credit 60': '60',
            'TW_Credit 10': '10',
            'TW_D_Credit 600': '600',
            'TW_D_Credit 300': '300',
            'TW_D_Credit 240': '240',
            'TW_D_Credit 120': '120',
            'TW_D_Credit 60': '60',
            'TW_D_Credit 10': '10'
        }
        CN_priceplan = {
            'CN_D_Credit 100': '100',
            'CN_D_Credit 16': '注册包 (16)',
            'CN_D_Pay-as-you-go': '随收随付（月结）',
            'CN_D_Credit 60':'60'
        }
        parameter_id={
            'CN':'df609',
            'TW':'df607',
            'KR':'df492'
        }

        url_list = []
        file_name = []
        for country in list:
            output = pd.DataFrame()
            for index, data in eval(country).iterrows():
                id = data['UserID']
                hospitalname = data['UserName']
                creditplan = eval(country+'_priceplan')[data['PricePlan']]
                string = "include%EE%80%801%EE%80%80EQ%EE%80%80" + str(id)

                params = {
                    parameter_id[country]: string,
                    "ds672.hospital_name_{}".format(country): hospitalname,
                    "ds125.CreditPlan_{}".format(country): creditplan
                }
                json_params = json.dumps(params)
                encoded_params = urllib.parse.quote(json_params)
                url = country_web[country] + "?params=" + encoded_params
                url_list.append(url)
                file_name.append('[{}]_'.format(hospitalname) + str(eval('style_for_'+country)))
  
        output['file name'] = file_name
        output['url'] = url_list
        self.output=output
        return output
    def download_reports(self):
        self.lenght=len(self.output)
        self.counter=0
        for index,data in self.output.iterrows():
            self.driver.get(data['url'])
            try:
                element = WebDriverWait(self.driver, 50).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="body"]/div/div/div[1]/div[2]/div/div[1]/div[1]/div[3]/div/div'))
                )
            finally:
                keyboard.press_and_release('ctrl + p')
                time.sleep(3)
                keyboard.press_and_release('enter')
                time.sleep(3)
                keyboard.write(data['file name'])
                time.sleep(0.5)
                keyboard.press_and_release('enter')
                self.counter+=1
                print('{a}/{b}'.format(a=self.counter,b=self.lenght))

if __name__='__main__':
    re=Reports()
    data=re.filter_users()
    re.get_repert_link()
    re.download_reports()
